In [ ]:
# Configuration : use this command to run pyspark shell rather than just pyspark command
pyspark --packages org.apache.hudi:hudi-spark$SPARK_VERSION-bundle_2.12:0.14.1 \
        --conf 'spark.serializer=org.apache.spark.serializer.KryoSerializer' \
        --conf 'spark.sql.catalog.spark_catalog=org.apache.spark.sql.hudi.catalog.HoodieCatalog' \
        --conf 'spark.sql.extensions=org.apache.spark.sql.hudi.HoodieSparkSessionExtension' \
        --conf 'spark.kryo.registrator=org.apache.spark.HoodieSparkKryoRegistrar'

In [2]:
"""
    Intended to write data in s3 with specific hudi schema and create hudi table in athena for quering purpose.
"""
                                                                                      
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize Spark session
spark = SparkSession.builder \
    .appName("HudiExample") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.hive.convertMetastoreParquet", "false") \
    .getOrCreate()

# Define schema
schema = StructType([
    StructField("id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("ts", StringType(), True)
])

# Create a sample dataframe
data = [("1", "Alice", 21, "2024-06-08T00:00:00Z"),
        ("2", "Bob", 22, "2024-06-08T00:00:00Z")]

df = spark.createDataFrame(data, schema)

# Write dataframe to Hudi table
hudi_options = {
    'hoodie.table.name': 'hudi_table',
    'hoodie.datasource.write.recordkey.field': 'id',
    'hoodie.datasource.write.partitionpath.field': '',
    'hoodie.datasource.write.table.name': 'hudi_table',
    'hoodie.datasource.write.operation': 'insert',
    'hoodie.upsert.shuffle.parallelism': 2,
    'hoodie.insert.shuffle.parallelism': 2
}

output_path = "s3://avengersmagicbucket/hudi_practice/"
print("write in progresss")
df.write.format("hudi").options(**hudi_options).mode("overwrite").save(output_path)
print("write completed")

Py4JJavaError: An error occurred while calling None.org.apache.spark.sql.SparkSession.
: org.apache.hudi.exception.HoodieException: Unable to load class
	at org.apache.hudi.common.util.ReflectionUtils.lambda$getClass$0(ReflectionUtils.java:55)
	at java.base/java.util.concurrent.ConcurrentHashMap.computeIfAbsent(ConcurrentHashMap.java:1705)
	at org.apache.hudi.common.util.ReflectionUtils.getClass(ReflectionUtils.java:51)
	at org.apache.hudi.common.util.ReflectionUtils.loadClass(ReflectionUtils.java:62)
	at org.apache.spark.sql.hudi.analysis.HoodieAnalysis$.customOptimizerRules(HoodieAnalysis.scala:166)
	at org.apache.spark.sql.hudi.HoodieSparkSessionExtension.apply(HoodieSparkSessionExtension.scala:43)
	at org.apache.spark.sql.hudi.HoodieSparkSessionExtension.apply(HoodieSparkSessionExtension.scala:28)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$2(SparkSession.scala:1370)
	at org.apache.spark.sql.SparkSession$.$anonfun$applyExtensions$2$adapted(SparkSession.scala:136